In [13]:
import tensorflow as tf
import numpy as np

In [5]:
# 사용가능한 gpu 목록 체크
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15538557232762965983,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16602457448685033596
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18159314504234412714
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10793441690
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7814919970016964625
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5"]

In [6]:
# GPU test
tf.debugging.set_log_device_placement(True)
# 텐서 생성
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Physical devices cannot be modified after being initialized


In [11]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

Virtual devices cannot be modified after being initialized


In [16]:
# load dataset
train_dataset = np.load('train_dataset.npz')
valid_dataset = np.load('valid_dataset.npz')
train_X, train_Y = train_dataset['X'],train_dataset['Y']
valid_X, valid_Y = valid_dataset['X'],valid_dataset['Y']
print(train_X.shape, train_Y.shape, valid_X.shape, valid_Y.shape)

(39537, 100, 30) (39537,) (4949, 100, 30) (4949,)


In [17]:
import numpy as np
import datetime as datetime
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import SimpleRNN, Input, Embedding, Dense, LSTM, LeakyReLU, LayerNormalization, BatchNormalization, Bidirectional
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [25]:
# label encoding
le = LabelEncoder()
le.fit(train_Y)
#print(le.classes_)
print(train_Y[:10])
train_Y_le = le.transform(train_Y)
valid_Y_le = le.transform(valid_Y)
print(train_Y_le[:10])

['편안/안정' '편안/안정' '편안/안정' '편안/안정' '편안/안정' '행복/즐거움' '행복/즐거움' '편안/안정' '편안/안정'
 '편안/안정']
[3 3 3 3 3 4 4 3 3 3]


In [26]:
# ohe
train_Y_ohe = to_categorical(train_Y_le)
valid_Y_ohe = to_categorical(valid_Y_le)
print(valid_Y_ohe[:10])

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [29]:
# solit train, test dataset
train_X, test_X, train_Y_ohe, test_Y_ohe = train_test_split(train_X, train_Y_ohe, test_size=0.2, random_state=2021, shuffle=False)

In [30]:
print(train_X.shape, test_X.shape, train_Y_ohe.shape, test_Y_ohe.shape)

(31629, 100, 30) (7908, 100, 30) (31629, 6) (7908, 6)


In [44]:
# build model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(6, activation='softmax'))

model.build(input_shape=(None,100,30))
model.summary()

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                multiple                  8064      
_________________________________________________________________
dense_1 (Dense)              multiple                  198       
Total params: 8,262
Trainable params: 8,262
Non-trainable params: 0
_________________________________________________________________


In [45]:
# set early stopping option & model checkpoint option
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [47]:
history = model.fit(train_X, train_Y_ohe, batch_size=128, epochs=1000, callbacks=[es, mc], validation_data=(valid_X, valid_Y_ohe))

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Train on 31629 samples, validate on 4949 samples
Epoch 1/1000
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

In [48]:
# 저장된 best_model로 test

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(test_X, test_Y_ohe)[1]))

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_283505 in device /job:localhost/replica:0/task:0/device:GPU:0
7908/1 [===========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

AttributeError: 'LabelEncoder' object has no attribute 'help'